In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g

from gsheet_config import HOTEL_CENSUS_INPUT_GSHEET_ID, HOTEL_CENSUS_OUTPUT_GSHEET_ID, HOTEL_CENSUS_SHEET_NAME, HOTEL_CENSUS_SHEET_RANGE
# gsheet_config in .gitignore for the moment

## Integration with Google Sheets API

Built from a few resources below
* https://developers.google.com/sheets/api/quickstart/python
* https://developers.google.com/sheets/api/guides/authorizing

Note: There aren't any tokens and credentials included in this repo. Rather, `*pickle` and `*credentials.json` reference local files. Will build secure way for app to get access to these files when pushing into production.

In [ ]:
def get_creds(personal_acct_flg = True): 
    # personal_acct_flg arg is for dev purposes. used personal gmail acct to develop.
    # will delete when pushing into prod
    
    token_filename = 'uorc_token.pickle' if personal_acct_flg is False else 'personal_token.pickle'
    creds_filename = 'uorc_credentials.json' if personal_acct_flg is False else 'personal_credentials.json'
    
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets', #edit, create, delete, view
        #'https://www.googleapis.com/auth/spreadsheets.readonly' # view only 
    ] 
    
    # The file represented by variable token_filename stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    creds = None
    if os.path.exists(token_filename):
        with open(token_filename, 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                creds_filename, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_filename, 'wb') as token:
            pickle.dump(creds, token)
    
    return creds

In [ ]:
def get_google_sheet(spreadsheet_id, sheet_range, creds):
          
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    g_sheet = service\
        .spreadsheets()\
        .values()\
        .get(spreadsheetId=spreadsheet_id, range=sheet_range).execute()
                                
    return g_sheet

In [ ]:
def gsheet_to_df(gsheet, header_row_idx = 0):
    
    ws = gsheet.get('values', [])
    if ws:
        header = ws[header_row_idx]
        values = ws[header_row_idx + 1:]
        
    df = pd.DataFrame(data=values, columns=header)
    
    return df

In [ ]:
gsheet_creds = get_creds(False) # boolean arg is for dev purposes. used personal gmail acct to develop.
hc_gsheet = get_google_sheet(HOTEL_CENSUS_INPUT_GSHEET_ID, HOTEL_CENSUS_SHEET_RANGE, gsheet_creds)
hc_df = gsheet_to_df(hc_gsheet)

## Validate input sheets

For each input sheet, we will want to perform validation that the new data is populated and formatted appropriately. `validate_input_hotel_census` will perform this validation.

In [ ]:
def validate_input_hotel_census(df):
    '''
    this function will include all necessary checks to ensure input hotel census data has everything needed for dashboard.
    
    will add functionality as we get more data. Checks include:
    - create primary key for each row (shift/date/hotel). 
    - can check output gsheet to see which rows are already included in output, and only perform validation on new rows
    - each row has required columns filled
    - each row has column values with appropriate data type.
    - send report/email with rows that fail validation check
    '''
    
    # right now fxn just returns copy of orig df. will build out soon
    validated_df = df.copy()
    return validated_df

In [ ]:
#change

In [ ]:
hc_df_validated = validate_input_hotel_census(hc_df)

## Upload to output Google Sheet

Once validation is performed, we can now upload into appropriate Google Drive to flow into dashboard.

*Note:* This step is not yet working. Need to build out authorization for service account to ensure creds are valid

In [ ]:
def send_to_output_gsheet(df, output_gsheet_id, output_ws_name, creds):
    '''
    putting in sep function to allow for validation that upload executed successfully (to be developed)
    
    currently not working
    '''
    success_upload = None
    
    d2g.upload(df, output_gsheet_id, output_ws_name, credentials=creds, row_names=False)
    
    success_upload = True
    
    return success_upload